In [1]:
import evebox as box
from tqdm.notebook import tqdm, trange

Reload


In [33]:
import importlib
#importlib.reload(box.esi)
importlib.reload(box.universe)
importlib.reload(box)

<module 'evebox' from 'D:\\Daten\\Repos\\tf_playground\\eveplan\\evebox\\__init__.py'>

In [35]:
box.Universe.from_esi(cache = 'universe.txt', tqdm = tqdm)